In [5]:
import os
import openai
import cv2
from deepface import DeepFace
import sounddevice as sd
from scipy.io.wavfile import write

import json
json_file = open("key.json")
json_data = json.load(json_file)
api_key = json_data["key"]
openai.api_key = api_key

''' 
To save code to GitHub:
Command Prompt -> Make sure in correct folder - cd Coding Lessons
git add gpt_api.ipynb
git commit -m "Whatever change made"
git push
'''

' \nTo save code to GitHub:\nCommand Prompt -> Make sure in correct folder - cd Coding Lessons\ngit add gpt_api.ipynb\ngit commit -m "Whatever change made"\ngit push\n'

In [10]:
def pureWebcam():
    vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)
    frameNumber = 30
   
   
    papapy = 0
    cv2.imshow("Tester", vid.read()[1])  
    def mouse_callback(event, x, y, flags, params):
        global papapy
        if(event == cv2.EVENT_LBUTTONDOWN):
            print(papapy)
            papapy += 1000
    cv2.setMouseCallback('Tester', mouse_callback)


    while True:
        # Get current frame from webcam - vid.read() returns 2 things, _, is used for the things we don't care about
        _, frame = vid.read()
        cv2.putText(frame, str(papapy), (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (255, 0, 0), 2)
        papapy += 1
        cv2.imshow("Tester", frame)


        # Press q to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    vid.release()
    cv2.destroyAllWindows()
pureWebcam()
       



28202
29202
30202
31202
32202
33202


In [2]:
def recordAudio():
    duration = 5 # seconds
    sample_rate = 44100
    input = 5 # microphone
    output = 8 # audio device

    print("Recording")
    myrecording = sd.rec(int(duration * sample_rate), samplerate = sample_rate, channels = 2)
    sd.wait()
    print("Not Recording")

    write('recording.wav', sample_rate, myrecording)
    audio_file = open("recording.wav", "rb")
    transcript = openai.Audio.transcribe("whisper-1", audio_file)

    return transcript


In [3]:
def openWebcamWindow():

    # Set up webcam capture
    vid = cv2.VideoCapture(0, cv2.CAP_DSHOW)

    frameNumber = 30
    emotionList = []
    x = 0

    blueColor = (0, 0, 255)
    redColor = (255, 0, 0)
    greenColor = (0, 255, 0)
    thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX
    recordingText = ""
    global myMessage
    global chatGPTMessage
    cv2.imshow("Facial Detection", vid.read()[1])
    
    def mouse_callback(event, x, y, flags, params):
        if(event == cv2.EVENT_LBUTTONDOWN and x <= 100 and y <= 100):
            global ra
            ra = recordAudio()
            recordingText = "RECORDING"
        if(event == cv2.EVENT_LBUTTONDOWN and (x >= 120 and x <= 220) and y <= 100):
            print(ra)
            rec = True
            recordingText = ""
            chat_completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "system", "content": "You are a helpful assistant."}, 
                                                                                {"role": "user", "content": ra["text"]}])
            myMessage = "Your Message: " + ra["text"]
            chatGPTMessage = "ChatGPT: " + chat_completion["choices"][0]["message"]["content"]
            
            
    # On mouse click, run a method             \/ this one
    cv2.setMouseCallback('Facial Detection', mouse_callback)

    while True:
        # Get current frame from webcam - vid.read() returns 2 things, _, is used for the things we don't care about
        _, frame = vid.read()
        x = 0

        analytics = DeepFace.analyze(frame, actions = ['emotion'], enforce_detection=False, silent=True)
        
        for x in range(len(analytics)):
            
            if(len(emotionList) > 30):
                emotionList.pop(len(emotionList)-1) # Removes last emotion from the list

            emotionList.insert(0, analytics[x]['dominant_emotion']) # Adds dominant emotion to list
            
            mainEmotion = "" 
            mostCommonEmotion = max(set(emotionList), key=emotionList.count)
            mainEmotion = mostCommonEmotion
                
            startingPoint = (analytics[x]['region']['x'], analytics[x]['region']['y'])
            endingPoint = (analytics[x]['region']['x'] + analytics[x]['region']['w'], analytics[x]['region']['y'] + analytics[x]['region']['h'])
            cv2.rectangle(frame, startingPoint, endingPoint, blueColor, thickness)
            cv2.putText(frame, mainEmotion, startingPoint, font, 1, redColor, thickness)
            cv2.rectangle(frame, (0, 0), (100, 100), greenColor, thickness)
            cv2.rectangle(frame, (120, 0), (220, 100), greenColor, thickness)
        
        cv2.putText(frame, myMessage, (250, 75), font, 1/2, redColor, 2)
        cv2.putText(frame, chatGPTMessage, (250, 125), font, 1/2, redColor, 2)
            
        
        # Display current frame
        cv2.imshow("Facial Detection", frame)
        x += 1
        
        # Press q to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    vid.release()
    cv2.destroyAllWindows()

openWebcamWindow()

NameError: name 'myMessage' is not defined

: 

In [7]:
def FaceAnalyze(img):
    analytics = DeepFace.analyze(img)
    print(analytics)
    print(analytics[0]['region']['x'])

FaceAnalyze('babyface.jpeg')

Action: emotion:   0%|          | 0/4 [00:00<?, ?it/s]

Action: race: 100%|██████████| 4/4 [00:00<00:00,  6.68it/s]  

[{'emotion': {'angry': 0.001027339264808421, 'disgust': 9.338208714727861e-10, 'fear': 0.0023542822912443117, 'happy': 0.05763810238692775, 'sad': 0.1472810720335586, 'surprise': 0.08147145757840321, 'neutral': 99.71022604168809}, 'dominant_emotion': 'neutral', 'region': {'x': 91, 'y': 58, 'w': 83, 'h': 83}, 'age': 19, 'gender': {'Woman': 0.6382221821695566, 'Man': 99.36177730560303}, 'dominant_gender': 'Man', 'race': {'asian': 6.2889449298381805, 'indian': 3.5093825310468674, 'black': 1.968146301805973, 'white': 51.9415020942688, 'middle eastern': 19.873665273189545, 'latino hispanic': 16.418352723121643}, 'dominant_race': 'white'}]
91


In [32]:
def faceDetection(img):

    # Get the image from img
    img = cv2.imread(img)
    
    # Shrink images
    img = cv2.resize(img, (0,0), fx=0.5, fy=0.5)

    analytics = DeepFace.analyze(img, actions = ['emotion'])

    blueColor = (0, 0, 255)
    redColor = (255, 0, 0)
    thickness = 3
    font = cv2.FONT_HERSHEY_SIMPLEX

    for x in range(len(analytics)):
        
        startingPoint = (analytics[x]['region']['x'], analytics[x]['region']['y'])
        endingPoint = (analytics[x]['region']['x'] + analytics[x]['region']['w'], analytics[x]['region']['y'] + analytics[x]['region']['h'])
        cv2.rectangle(img, startingPoint, endingPoint, blueColor, thickness)
        cv2.putText(img, analytics[x]['dominant_emotion'], startingPoint, font, 1, redColor, thickness)

    # Displays the image
    cv2.imshow('ffaaccee', img)
 
    cv2.waitKey(0)
    cv2.destroyAllWindows()

faceDetection('manyFaces.jpeg')


Action: emotion: 100%|██████████| 1/1 [00:00<00:00, 15.65it/s]


In [8]:
def openImage(img):

    # Get the image from img
    img = cv2.imread(img)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Only include pixels in the range of the color
    mask = cv2.inRange(hsv, (22, 250, 250), (24, 260, 260))
    inverted_mask = cv2.bitwise_not(mask) #change
    
    # Apply the mask on hsv
    hsv_mask = cv2.bitwise_and(img, img, mask=inverted_mask)#change
    
    imageWidth = img.shape[0] // 2
    imageHeight = img.shape[1] // 2

    greenColor = (0, 255, 0)
    startingPoint = (imageHeight - 10, imageWidth - 10)
    endingPoint = (imageHeight + 10, imageWidth + 10)
    thickness = -1

    cv2.rectangle(img, startingPoint, endingPoint, greenColor, thickness)

    # Displays the image
    #cv2.imshow('original image', img)
    #cv2.imshow('with mask',hsv_mask)
    cv2.imshow("Image with green box", img)
    
    

    def mouse_callback(event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            print(hsv[y,x])

    cv2.setMouseCallback('Image with green box', mouse_callback)


    cv2.waitKey(0)
    cv2.destroyAllWindows()

openImage('C:/Users/brody/Pictures/Camera Roll/DerpyTree.jpeg')
